In [1]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import torch
class bertWordEmbedding():
    def __init__(self,toCUDA):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased') 
        self.toCUDA =toCUDA
        print('create model...')
        if toCUDA ==1 :
            if torch.cuda.is_available():
                print('to CUDA')
                self.model.to('cuda')
            else:
                print('Hey man,you device not support CUDA, go back to use CPU')
                
        print('create end')
        print('How to use?')
        print('Format:  class.word2vec("Your text",mode)')
        print('Mode:1 sum last 4 hidden state' )
        print('Mode 2: concat last 4 hidden state')
        print('It will return (wordEmbeddings([mode1]n*768 or [mode2]n*(768*4)),sentenceEmbeddings(1*768)')
        
    def word2vec(self,text,mode):
        print('analyzing...')
        text = '[SEP] '+text+' [CLS]'
        
        tokenizedtext = self.tokenizer.tokenize(text)
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenizedtext)
        tokens_tensor = torch.tensor([indexed_tokens])
         
        segments_tensors = torch.tensor([1]*len([tokenizedtext]))
        
        
        with torch.no_grad():
            if self.toCUDA ==1:
                tokens_tensor=tokens_tensor.to('cuda')
                segments_tensors=segments_tensors.to('cuda')
            encoded_layers,_= self.model(tokens_tensor,segments_tensors)
        
        print('Get layers hidden done')
        token_embeddings = torch.stack(encoded_layers, dim=0)
        token_embeddings = torch.squeeze(token_embeddings,dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)
        
        token_vec=[]
        if mode==1:
            #sum last 4: mode 1
            for token in token_embeddings:
                sm = torch.sum(token[-4:],dim=0)
                token_vec.append(sm)
            token_vec = token_vec[1:-1]
            print('wordEmbeddings(sum last 4) result size is:{} words , size:{}*{} '.format(len(token_vec),len(token_vec),len(token_vec[0])))  
        elif mode == 2:
             #cat
            for token in token_embeddings:
                cat = torch.cat((token[-1],token[-2],token[-3],token[-4]),dim=0)
                token_vec.append(cat)
                
            token_vec = token_vec[1:-1]
            print('wordEmbeddings(cat) result size is:{} words , size:{}*{} '.format(len(token_vec),len(token_vec),len(token_vec[0])))  
        
        
        sequence_vecs = encoded_layers[11][0]
        # Calculate the average of all 22 token vectors.
        sentence_embedding = torch.mean(sequence_vecs, dim=0)
        print('sentenceEmbeddings result size is:{}'.format(len(sentence_embedding)))
        print('done')
        
        
        return (token_vec,sentence_embedding)

In [ ]:
superman = bertWordEmbedding(1)


create model...


In [ ]:
a = superman.word2vec('Hello world',1)
a

In [ ]:
print(len(a[0][0]))